In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alx-movie-recommendation-project-2024/sample_submission.csv
/kaggle/input/alx-movie-recommendation-project-2024/movies.csv
/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv
/kaggle/input/alx-movie-recommendation-project-2024/train.csv
/kaggle/input/alx-movie-recommendation-project-2024/test.csv
/kaggle/input/alx-movie-recommendation-project-2024/tags.csv
/kaggle/input/alx-movie-recommendation-project-2024/links.csv


In [2]:
# Libraries for importing and loading data
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import os
#from textwrap import wrap
import re
#from wordcloud import WordCloud

# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import SVD
from surprise import Reader, Dataset
from surprise.model_selection import GridSearchCV, cross_validate


# Libraries used during sorting procedures.
#import operator # <-- Convienient item retrieval during iteration
#import heapq # <-- Efficient sorting of large lists
from time import time

# Setting global constants to ensure notebook results are reproducible

RANDOM_STATE = 25


import warnings
warnings.filterwarnings('ignore')


# Garbage Collector
import gc

In [3]:
test  = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/test.csv')
train = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/train.csv')

In [4]:
train.sort_values(by=['userId'], inplace= True)
train.head()

,userId,movieId,rating,timestamp
6308822,1,296,5.0,1147880044
3137042,1,27721,3.0,1147869115
2533005,1,665,5.0,1147878820
2524478,1,4308,3.0,1147868534
1946297,1,1250,4.0,1147868414


In [5]:
test.sort_values(by=['userId'], inplace= True)
test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [6]:
train_sampled = train.drop(columns= 'timestamp')

# Drop key_words column for the train dataset
ratings = train_sampled

In [7]:
# Instanciate a Reader object from surprise package
reader = Reader()

# Instanciate the dataset object from surprice package with the ratings table and reader object
data = Dataset.load_from_df(ratings, reader)

# Instantiate an SVD model from the surprice package
svd = SVD(n_epochs=40,n_factors=2000,init_std_dev=0.005,random_state=RANDOM_STATE,verbose=1)

# Instanciate and build the dataset with data object from surprise package
trainset = data.build_full_trainset()

# Train the SVD model
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39


In [8]:
gc.collect()

0

In [9]:
from sklearn.metrics import mean_squared_error

# Make predictions for the test set
predictions = []
for _, row in train_sampled.iterrows():
    uid = row['userId']
    iid = row['movieId']
    true_r = row['rating']
    pred = svd.predict(uid, iid, r_ui=true_r)
    predictions.append((true_r, pred.est))

# Separate the true ratings and the predicted ratings
true_ratings = [true_r for true_r, pred in predictions]
predicted_ratings = [pred for true_r, pred in predictions]

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
print(f'RMSE: {rmse}')

RMSE: 0.4353193268919024


In [10]:
import pickle

model_save_path = "svd_model.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(svd,file)

In [11]:
model_load_path = "svd_model.pkl"
with open(model_load_path,'rb') as file:
    unpickled_model = pickle.load(file)

In [12]:
# Extract the userIds and corresponding movieIds into a python list
userId  = test['userId'].values.tolist()
movieId = test['movieId'].values.tolist()

# Create a range of numbers which is the size of list created above
count = len(userId)
rating = []
ids = []

# Make rating predictions with the corresponding userIds and movieIds, accessed through indexing
for i in range(count):
    result = svd.predict(userId[i], movieId[i])[3]

    # Store the predicted rating
    rating.append(result)

    # Prepare, save userId and movieId for submission
    ids.append(str(userId[i]) + '_' + str(movieId[i]))

In [13]:
# Convert ids and ratings to dataframe
test_sub_df =  pd.DataFrame({'Id': ids, 'rating': rating})
test_sub_df.head()

,Id,rating
0,1_2011,3.241999
1,1_4144,4.167554
2,1_5767,3.714021
3,1_6711,4.044816
4,1_7318,2.918314


In [14]:
test_sub_df.shape

(5000019, 2)

In [15]:
gc.collect()

0

In [16]:
# Convert dataframe to csv file for submission.
test_sub_df.to_csv('Kaggle_submission.csv', index=False)